In [ ]:
import os
import sys
import logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # FATAL
logging.getLogger('tensorflow').setLevel(logging.DEBUG)

try:
    from google.colab import drive
    drive.mount('/content/drive')
    !pip install -q ruamel.yaml
    !pip install -q tensorboard-plugin-profile
    project_path = '/content/drive/MyDrive/Colab Projects/QuantumFlow'
except:
    project_path = os.path.expanduser('~/QuantumFlow')

In [ ]:
os.chdir(project_path)
sys.path.append(project_path)

import numpy as np
import tensorflow as tf
import tree

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import quantumflow

experiment = 'crazynet'
run_name = 'default_deriv'

base_dir = os.path.join(project_path, "experiments", experiment)
params = quantumflow.utils.load_yaml(os.path.join(base_dir, 'hyperparams.yaml'))[run_name]
run_dir = os.path.join(base_dir, run_name)

In [ ]:
dataset_train = quantumflow.instantiate(params['dataset_train'], run_dir=run_dir)
dataset_train.build()

dataset_validate = quantumflow.instantiate(params['dataset_validate'], run_dir=run_dir)
dataset_validate.build()

In [ ]:
tf.keras.backend.clear_session()
tf.random.set_seed(params['seed'])

model = quantumflow.instantiate(params['model'], run_dir=run_dir, dataset=dataset_train)
display(model.summary())

In [ ]:
steps = 100

def loss_fn(outputs, targets):
    return tf.reduce_mean(tf.square(outputs['kinetic_energy_density'] - targets['kinetic_energy_density'])) + 10*tf.reduce_mean(tf.square(outputs['derivative'] - targets['derivative']))


def validation_fn():
    outputs_validate = tree.map_structure(lambda out: [], dataset_validate.targets)

    for index in range(dataset_validate.dataset_size):
        features = tree.map_structure(lambda inp: inp[index:index+1], dataset_validate.features)    
        tree.map_structure_up_to(dataset_validate.targets, lambda pred, arr: arr.append(pred.numpy()), model(features), outputs_validate)

    outputs_validate = tree.map_structure_up_to(dataset_validate.targets, lambda arr: np.concatenate(arr), outputs_validate)

    return tree.map_structure(lambda pred, target: np.mean(np.abs(pred - target)), outputs_validate, dataset_validate.targets)


optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
        
losses = []
validation = tree.map_structure(lambda out: [], dataset_validate.targets)

import time
start_time = time.time()

In [ ]:
@tf.function
def step_fn(features, targets):
    with tf.GradientTape() as tape:
        outputs = model(features, training=True) 
        loss_value = loss_fn(outputs, targets)

    grads = tape.gradient(loss_value, model.trainable_weights)

    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    return loss_value


print(f"{'step':>10} {'loss':>20} {'kinetic_energy':>20} {'kinetic_energy_density':>25}")

for step in range(steps):
    
    index = np.random.randint(100)
    features = tree.map_structure(lambda inp: inp[index:index+1], dataset_train.features)
    targets = tree.map_structure(lambda inp: inp[index:index+1], dataset_train.targets)
    
    loss_value = step_fn(features, targets)
    
    if step % 1000 == 0:
        mean_abs_err = validation_fn()
        print(f"{step:10d} {loss_value:20f} {mean_abs_err['kinetic_energy']:20f} {mean_abs_err['kinetic_energy_density']:25f}")
        tree.map_structure_up_to(dataset_validate.targets, lambda err, arr: arr.append(err), mean_abs_err, validation)
        
    losses.append(loss_value.numpy())
    
    if len(losses) > 25000:
        optimizer.learning_rate.assign(optimizer.learning_rate*0.9999)
        
# kinetic_energy 0.00004
# kinetic_energy_density 0.006

In [ ]:
tf.config.experimental.get_memory_info('GPU:0')['peak']/1024**3

In [ ]:
x = np.float32(dataset_train.x[:, np.newaxis])
x_inputs = np.float32(dataset_train.x[::10, np.newaxis])
print(x.shape)
print(x_inputs.shape)

In [ ]:
def get_projection_matrix(x, x_inputs):
    n_dim = x.shape[-1]
    x_diff = np.sqrt(np.sum(np.square(np.expand_dims(x, axis=1) - np.expand_dims(x_inputs, axis=0)), axis=-1))
    
    projection_fn = np.zeros_like(x_diff)
    #print(projection_fn.shape)
    # for every x
    # get nearest n_dim+1 points
    # interpolate
    
    for x in range(1, 49):
        projection_fn[:, x] = np.concatenate([np.zeros((x-1)*10), np.linspace(0, 1, 11), np.linspace(0.9, 0, 10), np.zeros(500-21-(x-1)*10)], axis=0)
        
    projection_fn[:, 0] = np.concatenate([np.linspace(1, 0, 11), np.zeros(500-11)], axis=0)
    projection_fn[:, -1] = np.concatenate([np.zeros(480), np.linspace(0, 1, 11), np.ones(9)], axis=0)
    
    return projection_fn

y = np.zeros_like(x)
y[::10] = 1

projection_matrix = get_projection_matrix(x, x_inputs)

plt.figure(figsize=(20, 3))
plt.plot(projection_matrix)
plt.plot(y, 'k.')
plt.show()

In [ ]:
density = dataset_train.density[:1]
proj_coeff = np.matmul(density, projection_matrix)

plt.figure(figsize=(20, 3))
plt.plot(dataset_train.x, density.transpose())
plt.show()

In [ ]:
proj_coeff.shape

In [ ]:
projection_matrix.shape

In [ ]:
proj_density = np.matmul(proj_coeff, projection_matrix.transpose())

In [ ]:
proj_density.shape

In [ ]:
plt.figure(figsize=(20, 3))
plt.plot(dataset_train.x, proj_density.transpose())
plt.plot(dataset_train.x[5::10], proj_coeff.transpose(), 'k.')
plt.show()